In [5]:
import math
import sys
sys.path.append('../../python/')  
from periphery import constant
print(constant.INV)

0


In [22]:
class Technology:
    def __init__(self, node_nm=45, roadmap='HP', transistor_type='conventional'):
        self.node_nm = node_nm
        self.roadmap = roadmap
        self.transistor_type = transistor_type
        self.initialized = False
        self.params = {}
        self._initialize()

    def _initialize(self):
        if self.initialized:
            print("Warning: Already initialized!")
            return
        if self.transistor_type == 'conventional':
            if self.node_nm == 45 and self.roadmap == 'HP':
                vdd = 1.0
                vth = 0.18
                phyGateLength = 45e-9
                capIdealGate = 4e-10
                capFringe = 5e-10
                capOverlap = capIdealGate * 0.2 if self.node_nm >= 22 else 0.0

                # Junction cap model (from BSIM4)
                buildInPotential = 0.9
                cjd = 1e-3
                cjswd = 2.5e-10
                cjswgd = 0.5e-10
                mjd = 0.5
                mjswd = 0.33
                mjswgd = 0.33
                #/* Properties not used so far */
                capPolywire = 0.0;	#/* TO-DO: we need to find the values */

                capJunction = cjd / pow(1 + vdd / buildInPotential, mjd)
                capSidewall = cjswd / pow(1 + vdd / buildInPotential, mjswd)
                capDrainToChannel = cjswgd / pow(1 + vdd / buildInPotential, mjswgd)

                # Current arrays (μA/μm)
                currentOnNmos = [930e-6 - i * 2e-6 for i in range(0, 101, 10)]      # NMOS on-current
                currentOnPmos = [700e-6 - i * 1.5e-6 for i in range(0, 101, 10)]    # PMOS on-current
                currentOffNmos = [100e-9 + i * 0.5e-9 for i in range(0, 101, 10)]   # NMOS off-current
                currentOffPmos = [80e-9 + i * 0.4e-9 for i in range(0, 101, 10)]    # PMOS off-current

                # Interpolation to full 0-100
                currentOnNmos = self._interpolate_full(currentOnNmos)
                currentOnPmos = self._interpolate_full(currentOnPmos)
                currentOffNmos = self._interpolate_full(currentOffNmos)
                currentOffPmos = self._interpolate_full(currentOffPmos)

                self.params = {
                    'vdd': vdd,
                    'vth': vth,
                    'phyGateLength': phyGateLength,
                    'capIdealGate': capIdealGate,
                    'capFringe': capFringe,
                    'capOverlap': capOverlap,
                    'capJunction': capJunction,
                    'capSidewall': capSidewall,
                    'capDrainToChannel': capDrainToChannel,
                    'effectiveResistanceMultiplier': 1.0,  # could be adjusted based on design
                    'current_gmNmos': 1e-3,  # conductance for gm unit is μS/μm
                    'current_gmPmos': 0.8e-3,
                    'currentOnNmos': currentOnNmos,
                    'currentOnPmos': currentOnPmos,
                    'currentOffNmos': currentOffNmos,
                    'currentOffPmos': currentOffPmos,
                    'pnSizeRatio': 2.0,
                    'transistorType': self.transistor_type,   
                    'capPolywire': capPolywire,  
                    'featureSize': self.node_nm * 1e-9       
                }

            elif self.node_nm == 14 and self.roadmap == 'LSTP':
                vdd = 0.8
                vth = 0.45
                phyGateLength = 14e-9
                capIdealGate = 2.5e-10
                capFringe = 3e-10
                capOverlap = 0.0  # don't use overlap cap in finFET nodes

                buildInPotential = 0.9
                cjd = 1e-3
                cjswd = 2.5e-10
                cjswgd = 0.5e-10
                mjd = 0.5
                mjswd = 0.33
                mjswgd = 0.33
                #/* Properties not used so far */
                capPolywire = 0.0;	#/* TO-DO: we need to find the values */

                capJunction = cjd / pow(1 + vdd / buildInPotential, mjd)
                capSidewall = cjswd / pow(1 + vdd / buildInPotential, mjswd)
                capDrainToChannel = cjswgd / pow(1 + vdd / buildInPotential, mjswgd)

                currentOnNmos = [600e-6 - i * 2e-6 for i in range(0, 101, 10)]
                currentOnPmos = [500e-6 - i * 1.5e-6 for i in range(0, 101, 10)]
                currentOffNmos = [10e-9 + i * 0.1e-9 for i in range(0, 101, 10)]
                currentOffPmos = [8e-9 + i * 0.1e-9 for i in range(0, 101, 10)]

                currentOnNmos = self._interpolate_full(currentOnNmos)
                currentOnPmos = self._interpolate_full(currentOnPmos)
                currentOffNmos = self._interpolate_full(currentOffNmos)
                currentOffPmos = self._interpolate_full(currentOffPmos)

                self.params = {
                    'vdd': vdd,
                    'vth': vth,
                    'phyGateLength': phyGateLength,
                    'capIdealGate': capIdealGate,
                    'capFringe': capFringe,
                    'capOverlap': capOverlap,
                    'capJunction': capJunction,
                    'capSidewall': capSidewall,
                    'capDrainToChannel': capDrainToChannel,
                    'effectiveResistanceMultiplier': 1.2,
                    'current_gmNmos': 2.0e-3,
                    'current_gmPmos': 1.6e-3,
                    'currentOnNmos': currentOnNmos,
                    'currentOnPmos': currentOnPmos,
                    'currentOffNmos': currentOffNmos,
                    'currentOffPmos': currentOffPmos,
                    'capPolywire': capPolywire,  
                    'pnSizeRatio': 2.0
                }
            else:
                raise ValueError(f"Unsupported node {self.node_nm}nm or roadmap {self.roadmap}")

            self.initialized = True

    def _interpolate_full(self, base):
        'gemerate a full 0-100 array from a base array with 10 steps'
        full = [0.0] * 101
        for i in range(0, 101, 10):
            full[i] = base[i // 10]
        for i in range(1, 100):
            if i % 10 != 0:
                low = (i // 10) * 10
                high = low + 10
                alpha = (i - low) / 10
                full[i] = full[low] * (1 - alpha) + full[high] * alpha
        return full

    def get_param(self, name):
        return self.params.get(name, None)

    def interpolate_current(self, name, bias_index):
        'Interpolate current values based on bias index (0-100)'
        if name not in self.params:
            return None
        arr = self.params[name]
        i_low = int(bias_index)
        i_high = min(100, i_low + 1)
        alpha = bias_index - i_low
        return arr[i_low] * (1 - alpha) + arr[i_high] * alpha

    def print_summary(self):
        print(f"Tech Node: {self.node_nm}nm, Roadmap: {self.roadmap}")
        for k, v in self.params.items():
            if isinstance(v, list):
                print(f"{k}: {v[:3]} ... {v[-3:]}")
            else:
                print(f"{k}: {v:.3e}" if isinstance(v, float) else f"{k}: {v}")


In [25]:
def calculate_mos_gate_cap(width, tech):
    """
    calculate the gate capacitance of a transistor (including ideal gate capacitance, overlap, fringe, and polywire capacitance)

    """
    # use the effective width for FinFET or bulk CMOS
    if tech.get_param('featureSize') >= 22e-9 or tech.get_param('transistorType') != 'conventional':
        # Bulk CMOS
        width_eff = width
    else:
        # FinFET: convert width to number of fins
        width_scaled = width * (tech.get_param('PitchFin') / (2 * tech.get_param('featureSize')))
        num_fins = math.ceil(width_scaled / tech.get_param('PitchFin'))
        fin_surface_area = 2 * tech.get_param('heightFin') + tech.get_param('widthFin')
        width_eff = num_fins * fin_surface_area

    gate_cap = (tech.get_param('capIdealGate') + tech.get_param('capOverlap') + tech.get_param('capFringe')) * width_eff
    gate_cap += tech.get_param('phyGateLength') * tech.get_param('capPolywire')

    return gate_cap


In [26]:
tech45 = Technology(node_nm=45, roadmap='HP')
cap = calculate_mos_gate_cap(1e-6, tech45)
print(f"Gate Capacitance at 45nm: {cap:.4e} F")


Gate Capacitance at 45nm: 9.8000e-16 F


In [54]:
def calculate_logicgate_area(gateType, num_Input, width_NMOS, width_PMOS,height_Transistor_Region, tech):
    # if tech.get_param('featureSize') <= 14e-9:  # FinFET
    #     width_NMOS *= tech('PitchFin') / (2 * tech.get_param('featureSize'))
    #     width_PMOS *= tech('PitchFin') / (2 * tech.get_param('featureSize'))
    #     height_Transistor_Region *= (MAX_TRANSISTOR_HEIGHT_FINFET / MAX_TRANSISTOR_HEIGHT)

    ratio = width_PMOS / (width_PMOS + width_NMOS) if (width_PMOS + width_NMOS) != 0 else 0
    num_Folded_PMOS = num_Folded_NMOS = 1

    if tech.get_param('featureSize') >= 22e-9 or tech.get_param('transistorType') != 'conventional':
        #consider electrical width of PMOS and NMOS which is y direction
        # Bulk
        if ratio == 0:  # no PMOS
            maxwidth_PMOS = 0
            maxwidth_NMOS = height_Transistor_Region - (constant.MIN_POLY_EXT_DIFF + constant.MIN_GAP_BET_FIELD_POLY / 2) * 2 * tech.get_param('featureSize')
        elif ratio == 1:    # no NMOS
            maxwidth_PMOS = height_Transistor_Region - (constant.MIN_POLY_EXT_DIFF + constant.MIN_GAP_BET_FIELD_POLY / 2) * 2 * tech.get_param('featureSize')
            maxwidth_NMOS = 0
        else:
            temp = height_Transistor_Region - constant.MIN_GAP_BET_P_AND_N_DIFFS * tech.get_param('featureSize') - (constant.MIN_POLY_EXT_DIFF + constant.MIN_GAP_BET_FIELD_POLY / 2) * 2 * tech.get_param('featureSize')
            maxwidth_PMOS = ratio * temp
            maxwidth_NMOS = (1 - ratio) * temp 
        #consider physical width of PMOS and NMOS which is x direction
        unit_Width_x_P, heightRegionP = 0, 0
        if width_PMOS > 0:
            if width_PMOS <= maxwidth_PMOS:
                unit_Width_x_P = 2 * (constant.POLY_WIDTH + constant.MIN_GAP_BET_GATE_POLY) * tech.get_param('featureSize') #########################################I don't know why this is multiplied by 2
                heightRegionP = width_PMOS
            else:
                num_Folded_PMOS = math.ceil(width_PMOS / maxwidth_PMOS)
                unit_Width_x_P = (num_Folded_PMOS + 1) * (constant.POLY_WIDTH + constant.MIN_GAP_BET_GATE_POLY) * tech.get_param('featureSize') #########################################sane problem, it add one more unit width
                heightRegionP = maxwidth_PMOS

        unit_Width_x_N, heightRegionN = 0, 0
        if width_NMOS > 0:
            if width_NMOS <= maxwidth_NMOS:
                unit_Width_x_N = 2 * (constant.POLY_WIDTH + constant.MIN_GAP_BET_GATE_POLY) * tech.get_param('featureSize')
                heightRegionN = width_NMOS
            else:
                num_Folded_NMOS = math.ceil(width_NMOS / maxwidth_NMOS)
                unit_Width_x_N = (num_Folded_NMOS + 1) * (constant.POLY_WIDTH + constant.MIN_GAP_BET_GATE_POLY) * tech.get_param('featureSize')
                heightRegionN = maxwidth_NMOS

    else:
        # # FinFET
        # def calc_max_fin(ratio_part):
        #     return math.floor(
        #         ratio_part * (height_Transistor_Region - MIN_GAP_BET_P_AND_N_DIFFS * tech.get_param('featureSize') -
        #                       (MIN_POLY_EXT_DIFF + MIN_GAP_BET_FIELD_POLY / 2) * 2 * tech.get_param('featureSize')) / tech('PitchFin')
        #     ) + 1

        # maxNumPFin, maxNumNFin = 0, 0
        # if ratio == 0:
        #     maxNumNFin = calc_max_fin(1)
        # elif ratio == 1:
        #     maxNumPFin = calc_max_fin(1)
        # else:
        #     maxNumPFin = calc_max_fin(ratio)
        #     maxNumNFin = calc_max_fin(1 - ratio)

        # unit_Width_x_P, heightRegionP = 0, 0
        # NumPFin = math.ceil(width_PMOS / tech('PitchFin'))
        # if NumPFin > 0:
        #     if NumPFin <= maxNumPFin:
        #         unit_Width_x_P = 2 * (POLY_WIDTH_FINFET + MIN_GAP_BET_GATE_POLY_FINFET) * tech.get_param('featureSize')
        #         heightRegionP = (NumPFin - 1) * tech('PitchFin') + tech('widthFin')
        #     else:
        #         num_Folded_PMOS = math.ceil(NumPFin / maxNumPFin)
        #         unit_Width_x_P = (num_Folded_PMOS + 1) * (POLY_WIDTH_FINFET + MIN_GAP_BET_GATE_POLY_FINFET) * tech.get_param('featureSize')
        #         heightRegionP = (maxNumPFin - 1) * tech('PitchFin') + tech('widthFin')

        # unit_Width_x_N, heightRegionN = 0, 0
        # NumNFin = math.ceil(width_NMOS / tech('PitchFin'))
        # if NumNFin > 0:
        #     if NumNFin <= maxNumNFin:
        #         unit_Width_x_N = 2 * (POLY_WIDTH_FINFET + MIN_GAP_BET_GATE_POLY_FINFET) * tech.get_param('featureSize')
        #         heightRegionN = (NumNFin - 1) * tech('PitchFin') + tech('widthFin')
        #     else:
        #         num_Folded_NMOS = math.ceil(NumNFin / maxNumNFin)
        #         unit_Width_x_N = (num_Folded_NMOS + 1) * (POLY_WIDTH_FINFET + MIN_GAP_BET_GATE_POLY_FINFET) * tech.get_param('featureSize')
        #         heightRegionN = (maxNumNFin - 1) * tech('PitchFin') + tech('widthFin')
        print("FinFET support is not implemented in this version.")

    # # gate type width computation
    # def shared_region_correction(unit_width, isFinFET=False):
    #     if isFinFET:
    #         poly_width = POLY_WIDTH_FINFET + MIN_GAP_BET_GATE_POLY_FINFET
    #     else:
    #         poly_width = POLY_WIDTH + MIN_GAP_BET_GATE_POLY
    #     return unit_width * num_Input - (num_Input - 1) * tech.get_param('featureSize') * poly_width

    # # gate type width computation
    if gateType == constant.INV:
        width_x_P = unit_Width_x_P
        width_x_N = unit_Width_x_N
    elif gateType in (constant.NOR, constant.NAND):
        isFinFET = tech.get_param('featureSize') < 22e-9 and tech.get_param('transistorType') == 'conventional'
        if num_Folded_PMOS == 1 and num_Folded_NMOS == 1:   # no folding
            width_x_P = unit_Width_x_P/2 * (num_Input+1)
            width_x_N = unit_Width_x_N/2 * (num_Input+1)
        else:
            width_x_P = unit_Width_x_P/2 * (num_Input * num_Folded_PMOS+1)
            width_x_N = unit_Width_x_N/2 * (num_Input * num_Folded_NMOS+1)
    else:
        width_x_P = width_x_N = 0

    print("maxtransistor height:", height_Transistor_Region)
    print("maxwidth_PMOS:", maxwidth_PMOS)
    print("maxwidth_NMOS:", maxwidth_NMOS)
    # print("unit_Width_x_P:", unit_Width_x_P)
    # print("unit_Width_x_N:", unit_Width_x_N)
    # print("num_Input:", num_Input)
    print("num_Folded_PMOS:", num_Folded_PMOS)
    # print(unit_Width_x_P/2)
    # print((num_Input+1))
    # print(unit_Width_x_N/2 * (num_Input+1))
    # print("width_x_P:", width_x_P)
    # print("width_x_N:", width_x_N)
    width_x = max(width_x_P, width_x_N)
    height_y = height_Transistor_Region
    return width_x, height_y, width_x * height_y


In [57]:
width_nmos = 1e-6  # 1µm
width_pmos = 1e-6  # 2µm

height_out_y = [0]
width_out_x = [0]

w_nand, h_nand, area_nand = calculate_logicgate_area(constant.NAND, 2, width_nmos, width_pmos,
                                      constant.MAX_TRANSISTOR_HEIGHT*tech45.get_param('featureSize'), tech45 )
w_inv, h_inv, area_inv = calculate_logicgate_area(constant.INV, 1, width_nmos, width_pmos,
                                        constant.MAX_TRANSISTOR_HEIGHT*tech45.get_param('featureSize'), tech45 )

print(f"NAND Width: {w_nand:.2e} m")
print(f"NAND Height: {h_nand:.2e} m")
print(f"NAND Area: {area_nand:.2e} m²")
print(f"INV Width: {w_inv:.2e} m")
print(f"INV Height: {h_inv:.2e} m")
print(f"INV Area: {area_inv:.2e} m²")

maxtransistor height: 1.2600000000000002e-06
maxwidth_PMOS: 4.702500000000001e-07
maxwidth_NMOS: 4.702500000000001e-07
num_Folded_PMOS: 3
maxtransistor height: 1.2600000000000002e-06
maxwidth_PMOS: 4.702500000000001e-07
maxwidth_NMOS: 4.702500000000001e-07
num_Folded_PMOS: 3
NAND Width: 2.39e-06 m
NAND Height: 1.26e-06 m
NAND Area: 3.02e-12 m²
INV Width: 6.84e-07 m
INV Height: 1.26e-06 m
INV Area: 8.62e-13 m²


In [68]:
def calculate_logicgate_cap(gate_type, num_Input, width_NMOS, width_PMOS, height_transistor_region, tech):
    
    ratio = width_PMOS / (width_PMOS + width_NMOS) if (width_PMOS + width_NMOS) > 0 else 0
    num_folded_pmos = num_folded_nmos = 1
    # FinFET adjustment
    if tech.get_param('featureSize') <= 14e-9:
        width_NMOS *= tech('PitchFin') / (2 * tech.get_param('featureSize'))
        width_PMOS *= tech('PitchFin') / (2 * tech.get_param('featureSize'))
        height_transistor_region *= (34 / 28)

    #consider electrical width of PMOS and NMOS which is y direction
    if tech.get_param('featureSize') >= 22e-9 or tech.get_param('transistorType') != 'conventional':  
        # Bulk
        if ratio == 0:  # no PMOS
            max_width_pmos = 0
            max_width_nmos = height_transistor_region - (constant.MIN_POLY_EXT_DIFF + constant.MIN_GAP_BET_FIELD_POLY / 2)* 2 * tech.get_param('featureSize')
        elif ratio == 1:    # no NMOS
            max_width_pmos = height_transistor_region - (constant.MIN_POLY_EXT_DIFF + constant.MIN_GAP_BET_FIELD_POLY / 2) * 2 * tech.get_param('featureSize')
            max_width_nmos = 0
        else:
            temp = height_transistor_region - constant.MIN_GAP_BET_P_AND_N_DIFFS * tech.get_param('featureSize') - (constant.MIN_POLY_EXT_DIFF + constant.MIN_GAP_BET_FIELD_POLY / 2) * 2 * tech.get_param('featureSize')
            maxwidth_PMOS = ratio * temp
            maxwidth_NMOS = (1 - ratio) * temp 
            
        #consider physical width of PMOS and NMOS which is x direction\
        if width_PMOS > 0:
            if width_PMOS <= maxwidth_PMOS:
                unit_width_Drain_P = constant.MIN_GAP_BET_GATE_POLY * tech.get_param('featureSize') 
                unit_Width_Source_P = unit_width_Drain_P;
                height_Drain_P = width_PMOS
            else:
                num_Folded_PMOS = math.ceil(width_PMOS / maxwidth_PMOS)
                unit_width_Drain_P = math.ceil((num_Folded_PMOS+1)/2) * constant.MIN_GAP_BET_GATE_POLY * tech.get_param('featureSize') 
                unit_width_Source_P = math.floor((num_Folded_PMOS+1)/2) * constant.MIN_GAP_BET_GATE_POLY * tech.get_param('featureSize') 
                height_Drain_P = maxwidth_PMOS
        else:
            unit_width_Drain_P = unit_Width_Source_P = height_Drain_P = 0

        if width_NMOS > 0:
            if width_NMOS <= maxwidth_NMOS:
                unit_width_Drain_N = constant.MIN_GAP_BET_GATE_POLY * tech.get_param('featureSize')
                unit_Width_Source_N = unit_width_Drain_N
                height_Drain_N = width_NMOS
            else:
                num_Folded_NMOS = math.ceil(width_NMOS / maxwidth_NMOS)
                unit_width_Drain_N = math.ceil((num_Folded_NMOS + 1) / 2) * tech.get_param('featureSize') * constant.MIN_GAP_BET_GATE_POLY
                unit_Width_Source_N = math.floor((num_Folded_NMOS + 1) / 2) * tech.get_param('featureSize') * constant.MIN_GAP_BET_GATE_POLY
                height_Drain_N = maxwidth_NMOS
        else:
            unit_width_Drain_N = unit_Width_Source_N = height_Drain_N = 0
    
    else:  # FinFET
        print("FinFET support is not implemented in this version.")
        # pitch = tech('PitchFin')
        # tech.get_param('featureSize') = tech.get_param('featureSize')
        # height_effective = height_transistor_region - (1.0 + 1.6 / 2) * 2 * tech.get_param('featureSize')
        # max_num_pfin = int((ratio * (height_effective - 3.5 * tech.get_param('featureSize'))) / pitch) + 1
        # max_num_nfin = int(((1 - ratio) * (height_effective - 3.5 * tech.get_param('featureSize'))) / pitch) + 1

        # num_pfin = math.ceil(width_pmos / pitch)
        # num_nfin = math.ceil(width_nmos / pitch)

        # def calc_drain_finfet(num_fin, max_fin, tech_val):
        #     if num_fin <= max_fin:
        #         unit_drain = tech_val
        #         unit_source = tech_val
        #         height_drain = (num_fin - 1) * pitch + tech('widthFin')
        #     else:
        #         folds = math.ceil(num_fin / max_fin)
        #         unit_drain = math.ceil((folds + 1) / 2) * tech_val
        #         unit_source = math.floor((folds + 1) / 2) * tech_val
        #         height_drain = (max_fin - 1) * pitch + tech('widthFin')
        #     return unit_drain, unit_source, height_drain, folds if num_fin > max_fin else 1

        # unit_width_drain_p, unit_width_source_p, height_drain_p, num_folded_pmos = calc_drain_finfet(
        #     num_pfin, max_num_pfin, tech.get_param('featureSize') * 3.9)
        # unit_width_drain_n, unit_width_source_n, height_drain_n, num_folded_nmos = calc_drain_finfet(
        #     num_nfin, max_num_nfin, tech.get_param('featureSize') * 3.9)

    # Gate-specific drain capacitance model (INV, NOR, NAND)
    if gate_type == constant.INV:
        if width_PMOS > 0:
            width_drain_p = unit_width_Drain_P * ((num_folded_pmos + 1) // 2)
            width_drain_sidewall_p = (unit_width_Drain_P + height_Drain_P) * 2 * ((num_folded_pmos+1) // 2)
        if width_NMOS > 0:
            width_drain_n = unit_width_Drain_N * ((num_folded_pmos + 1) // 2)
            width_drain_sidewall_n = (unit_width_Drain_N + height_Drain_N) * 2 * ((num_folded_nmos+1) // 2)
    elif gate_type == constant.NOR:
        if width_PMOS > 0:              #pmos only has one drain as output
            width_drain_p = unit_width_Drain_P * ((num_folded_pmos + 1) // 2)
            width_drain_sidewall_p = (unit_width_Drain_P + height_Drain_P) * 2 * ((num_folded_pmos+1) // 2)
        if width_NMOS > 0:              #nmos has all the drains as output
            width_drain_n = unit_width_Drain_N * ((num_folded_nmos * num_Input + 1) // 2)
            width_drain_sidewall_n = (unit_width_Drain_N + height_Drain_N) * 2 * ((num_folded_nmos * num_Input+1) // 2)
    elif gate_type == constant.NAND:
        if width_PMOS > 0:              #pmos has all the drains as output
            width_drain_p = unit_width_Drain_P * ((num_folded_pmos * num_Input + 1) // 2)
            width_drain_sidewall_p = (unit_width_Drain_P + height_Drain_P) * 2 * ((num_folded_pmos * num_Input+1) // 2)
        if width_NMOS > 0:              #nmos only has one drain as output
            width_drain_n = unit_width_Drain_N * ((num_folded_pmos + 1) // 2)
            width_drain_sidewall_n = (unit_width_Drain_N + height_Drain_N) * 2 * ((num_folded_nmos+1) // 2)
            


    cap_drain_bottom_p = width_drain_p * height_Drain_P * tech.get_param('capJunction')
    cap_drain_bottom_n = width_drain_n * height_Drain_N * tech.get_param('capJunction')
    cap_drain_side_p = width_drain_sidewall_p * tech.get_param('capSidewall')
    cap_drain_side_n = width_drain_sidewall_n * tech.get_param('capSidewall')
    cap_drain_channel_p = num_folded_pmos * height_Drain_P * tech.get_param('capDrainToChannel')
    cap_drain_channel_n = num_folded_nmos * height_Drain_N * tech.get_param('capDrainToChannel')

    cap_output = cap_drain_bottom_p + cap_drain_bottom_n + cap_drain_side_p + cap_drain_side_n + \
                 cap_drain_channel_p + cap_drain_channel_n
    cap_input = calculate_mos_gate_cap(width_nmos, tech) + calculate_mos_gate_cap(width_pmos, tech)

    return cap_input, cap_output


In [71]:
width_nmos = 1e-6  # 1µm
width_pmos = 1e-6  # 2µm

height_out_y = [0]
width_out_x = [0]

cap_input_nand, cap_output_nand  = calculate_logicgate_cap(constant.NAND, 2, width_nmos, width_pmos,
                                      constant.MAX_TRANSISTOR_HEIGHT*tech45.get_param('featureSize'), tech45 )
cap_input_inv, cap_output_inv = calculate_logicgate_cap(constant.INV, 1, width_nmos, width_pmos,
                                        constant.MAX_TRANSISTOR_HEIGHT*tech45.get_param('featureSize'), tech45 )
print(f"NAND Input Capacitance: {cap_input_nand:.2e} F")
print(f"NAND Output Capacitance: {cap_output_nand:.2e} F")
print(f"INV Input Capacitance: {cap_input_inv:.2e} F")
print(f"INV Output Capacitance: {cap_output_inv:.2e} F")

NAND Input Capacitance: 1.96e-15 F
NAND Output Capacitance: 7.64e-16 F
INV Input Capacitance: 1.96e-15 F
INV Output Capacitance: 7.64e-16 F


In [77]:
def calculate_logicgate_leakage(gate_type, num_input, width_nmos, width_pmos, temperature, tech):
    temp_index = int(temperature) - 300
    if temp_index > 100 or temp_index < 0:
        raise ValueError("Error: Temperature is out of range (300K to 400K)")

    leak_n = tech.get_param('currentOffNmos')
    leak_p = tech.get_param('currentOffPmos')

    if tech.get_param('featureSize') >= 22e-9 or tech.get_param('transistorType') != 'conventional':
        # Bulk CMOS
        width_nmos_eff = width_nmos
        width_pmos_eff = width_pmos
    else:  # FinFET
        width_nmos *= tech.get_param('PitchFin') / (2 * tech.get_param('featureSize'))
        width_pmos *= tech.get_param('PitchFin') / (2 * tech.get_param('featureSize'))
        width_nmos_eff = math.ceil(width_nmos / tech.get_param('PitchFin')) * (2 * tech.get_param('heightFin') + tech.get_param('widthFin'))
        width_pmos_eff = math.ceil(width_pmos / tech.get_param('PitchFin')) * (2 * tech.get_param('heightFin') + tech.get_param('widthFin'))

    if gate_type == constant.INV:  # INV
        leakage_n = width_nmos_eff * leak_n[temp_index]
        leakage_p = width_pmos_eff * leak_p[temp_index]
        return (leakage_n + leakage_p) / 2
    elif gate_type == constant.NOR:  # NOR
        leakage_n = width_nmos_eff * leak_n[temp_index] * num_input
        if num_input == 2:
            return constant.AVG_RATIO_LEAK_2INPUT_NOR * leakage_n
        else:
            return constant.AVG_RATIO_LEAK_3INPUT_NOR * leakage_n
    elif gate_type == constant.NAND: # NAND
        leakage_p = width_pmos_eff * leak_p[temp_index] * num_input
        if num_input == 2:
            return constant.AVG_RATIO_LEAK_2INPUT_NAND * leakage_p
        else:
            return constant.AVG_RATIO_LEAK_3INPUT_NAND * leakage_p
    else:
        return 0.0


In [82]:
temperature = 300  # Kelvin
leakage_nand  = calculate_logicgate_leakage(constant.NAND, 2, width_nmos, width_pmos,
                                      temperature, tech45 )
leakage_inv = calculate_logicgate_leakage(constant.INV, 1, width_nmos, width_pmos,
                                        temperature, tech45 )
print(f"NAND Leakage Current: {leakage_nand:.2e} A")
print(f"INV Leakage Current: {leakage_inv:.2e} A")

NAND Leakage Current: 7.68e-14 A
INV Leakage Current: 9.00e-14 A


In [86]:
def calculate_on_resistance(width, mos_type, temperature, tech):
    """
    calculate the on-resistance (R_on) of a MOS transistor given its width, type, and temperature
    returns the on-resistance in ohms (Ω)
    """
    temp_index = int(temperature) - 300
    if temp_index < 0 or temp_index > 100:
        raise ValueError("Temperature is out of range [300K, 400K]")

    # calculate effective width considering FinFET or bulk CMOS
    if tech.get_param('featureSize') >= 22e-9 or tech.get_param('transistorType') != 'conventional':
        # Bulk CMOS
        width_eff = width
    else:
        # FinFET: convert width to number of fins
        width *= tech('PitchFin') / (2 * tech.get_param('featureSize'))
        width_eff = math.ceil(width / tech('PitchFin')) * (2 * tech('heightFin') + tech('widthFin'))

    # based on lookup table for current on
    if mos_type == 0:  # NMOS
        I_on = tech.get_param('currentOnNmos')[temp_index]
    else:              # PMOS
        I_on = tech.get_param('currentOnPmos')[temp_index]

    # calculate the on-resistance
    resistance = tech.get_param('effectiveResistanceMultiplier') * tech.get_param('vdd') / (I_on * width_eff)
    return resistance


In [89]:
temperature = 300  # Kelvin
width_nmos = 1e-6  # 1µm
width_pmos = 2e-6  # 2µm
on_resistance_NMOS  = calculate_on_resistance(width_nmos, constant.NMOS,temperature, tech45 )
on_resistance_PMOS = calculate_on_resistance(width_pmos, constant.PMOS ,temperature, tech45 )
print(f"NMOS On-Resistance: {on_resistance_NMOS:.2e} Ω")
print(f"PMOS On-Resistance: {on_resistance_PMOS:.2e} Ω")

NMOS On-Resistance: 1.08e+09 Ω
PMOS On-Resistance: 7.14e+08 Ω


In [92]:
def calculate_transconductance(width, mos_type, tech):
    """
    calculate the transconductance (g_m) of a MOS transistor given its width and type
    returns the transconductance in siemens (S)
    """
    # assume Vgs is at 70% of Vdd for overdrive voltage calculation
    v_ov = 0.7 * tech.get_param('vdd') - tech.get_param('vth')  # effective overdrive voltage

    # gm = (2 * I_on) / V_ov
    if mos_type == 0:  # NMOS
        gm = (2 * tech.get_param('current_gmNmos')) * width / v_ov
    else:  # PMOS
        gm = (2 * tech.get_param('current_gmPmos')) * width / v_ov

    return gm


In [94]:
temperature = 300  # Kelvin
width_nmos = 1e-6  # 1µm
width_pmos = 2e-6  # 2µm
transconductance_NMOS = calculate_transconductance(width_nmos, constant.NMOS, tech45)
transconductance_PMOS = calculate_transconductance(width_pmos, constant.PMOS, tech45)
print(f"NMOS transconductance: {transconductance_NMOS:.2e} s")
print(f"PMOS transconductance: {transconductance_PMOS:.2e} s")

NMOS transconductance: 3.85e-09 s
PMOS transconductance: 6.15e-09 s


In [ ]:
def horowitz(tr, beta, ramp_input):
    """
    Horowitz delay model estimation.
    
    Parameters:
    tr (float): intrinsic delay of the gate (time constant, tau)
    beta (float): output response factor (usually set to 0.5 in simplified form)
    ramp_input (float): input signal ramp time (transition time)
    
    Returns:
    result (float): propagation delay
    ramp_output (float): estimated output ramp rate (1/result)

    t_delay = tr * sqrt(ln(vs)^2 + 2 * alpha * beta * (1 - vs))
    alpha = 1 / (ramp_input * tr) is the ramp factor
    beta is gate drive factor like gm/C
    VS is switching voltage, typically normalized to 0.5 in many models.
    """
    vs = 0.5  # Normalized switching voltage
    alpha = 1 / (ramp_input * tr)
    beta = 0.5  # Simplified model (used in CACTI and similar tools)

    result = tr * math.sqrt(math.log(vs)**2 + 2 * alpha * beta * (1 - vs))
    ramp_output = (1 - vs) / result if result != 0 else 0

    return result, ramp_output


In [ ]:
def nonlinear_resistance(R, NL, Vw, Vr, V):
    """
    calculate the nonlinear resistance based on the given parameters.
    parameters:
    R   : resistance at reference voltage Vr
    NL  : nonlinear coefficient (I(Vw/2)/I(V))
    Vw  : write voltage window (e.g., ±1V)
    Vr  : reference voltage (e.g., 0.1V)
    V   : actual applied voltage

    returns:
    R_NL : nonlinear resistance at voltage V
    """
    if V == 0:
        return float('inf')  # when V is zero, resistance is infinite
    exponent = (Vr - V) / (Vw / 2)
    R_NL = R * V / Vr * (NL ** exponent)
    return R_NL
